<a href="https://colab.research.google.com/github/SampMark/Computational-Thinking/blob/main/Debugging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exercício de Debugging**

O exercício proposto no curso **MITx 6.86x**: *Machine Learning with Python-From Linear Models to Deep Learning* tem por objetivo contribuir para o desenvolvimento de habilidades de **debugging** (depuração) em Python, especificamente usando o **pdb** (Python Debugger).  

Em suma, o foco é corrigir um código com bugs e garantir que funcione dentro do esperado. Neste caso, vamos trabalhar aqui, com sucessivas versões do código, até resolver o bug.

**Contexto do Exercício**

O exercício propõe a depuração de uma função chamada `get_sum_metrics`, que recebe recebe dois argumentos:

* `prediction`: Uma previsão (por exemplo, um valor numérico ou uma classificação).

* `metrics`: uma lista de métricas (funções) que devem ser aplicadas à previsão. Cada métrica é uma função que deve ser aplicada à predição como entrada e retornar um valor.

O objetivo da função é somar os resultados dessas métricas aplicadas à predição, juntamente com três métricas padrão que adicionam os valores 0, 1 e 2 à predição.

* **Natureza das métricas**: as métricas são funções e não números, então a implementação deve garantir que elas sejam chamadas corretamente.

* **Adição de valores padrão**: além das métricas fornecidas, a função também deve adicionar `prediction + 0`, `prediction + 1` e `prediction + 2` ao resultado final.

## **Possíveis problemas na versão inical do código**

---

Os seguintes problemas podem estar presentes no código original abaixo:

* **Chamadas incorretas das funções na lista de métricas**: Se a implementação não chamar as funções corretamente, ocorrerá um erro.
* **Somar funções em vez de valores numéricos**: se as funções na lista não forem chamadas antes da soma, a operação pode falhar.
* **Não inclusão das métricas padrão**: esquecimento de incluir as métricas padrão (0, 1 e 2).
* **Falta de tratamento de erros**: Se alguma métrica falhar ao processar a predição, o programa pode quebrar.

In [2]:
def get_sum_metrics(predictions, metrics=[]):
    for i in range(3):
        metrics.append(lambda x: x + i)

    sum_metrics = 0
    for metric in metrics:
        sum_metrics += metric(predictions)

    return sum_metrics

print(get_sum_metrics(0))

6


## **1. Debugging test** ❌

In [3]:
def get_sum_metrics(prediction, metrics):
    for i in range(3):
        metrics.append((lambda i: lambda x: x + i)(i))

    sum_metrics = 0
    for metric in metrics:
        sum_metrics += metric(prediction)

    return sum_metrics

print(get_sum_metrics(0, []))

3


## **2. Debugging test** ❌

In [4]:
def get_sum_metrics(predictions, metrics=[]):
    for i in range(3):
        metrics.append(lambda x, i=i: x + i)  # Captura o valor de i

    sum_metrics = 0
    for metric in metrics:
        sum_metrics += metric(predictions)

    return sum_metrics

print(get_sum_metrics(0, []))

3


## **3. Debugging test** ❌

In [5]:
def get_sum_metrics(predictions, metrics=None):
    if metrics is None:
        metrics = []

    for i in range(3):
        metrics.append(lambda x, i=i: x + i)  # Captura o valor correto de i

    sum_metrics = 0
    for metric in metrics:
        sum_metrics += metric(predictions)
        print(sum_metrics)  # Exibir resultado intermediário como no enunciado

    return sum_metrics

print(get_sum_metrics(0, []))

0
1
3
3


In [8]:
def test_get_sum_metrics():
    def metric1(x): return x * 2
    def metric2(x): return x - 3

    predictions = 3
    metrics = [metric1, metric2]

    print("✅Saída esperada:")
    print("3\n6\n9\n6\n11\n3\n6\n9")

    print("\n⚠️Saída obtida:")
    get_sum_metrics(predictions, metrics)

test_get_sum_metrics()

✅Saída esperada:
3
6
9
6
11
3
6
9

⚠️Saída obtida:
6
6
9
13
18


## **4. Debugging test** Pass! ✅

In [9]:
def get_sum_metrics(predictions, metrics=None):
    """
    Calcula a soma de métricas para uma determinada previsão.

    Argumentos:
    prediction: O valor da previsão.
    metrics: Uma lista de funções métricas para aplicar à previsão. Se None, apenas as métricas padrão serão usadas.

    Retorna:
    A soma de todas as métricas aplicadas à previsão.
    """
    if metrics is None:
        metrics = []

    # Adicionar métricas padrão (0, 1, 2)
    for i in range(3):
        metrics.append((lambda i: lambda x: x + i)(i))

    sum_metrics = 0

    # Aplicar todas as métricas (personalizadas e padrão)
    for metric_func in metrics:
        sum_metrics += metric_func(predictions)

    return sum_metrics

# Exemplo de uso
print(get_sum_metrics(0, []))  # Saída esperada: 3 (0 + 1 + 2)

3


In [10]:
def test_get_sum_metrics():
    """
    Testa a função get_sum_metrics com vários casos de teste e verifica se o resultado
    corresponde ao "Correct Output" esperado.
    """
    # Casos de teste com os resultados esperados
    test_cases = [
        (0, [], 3),          # prediction=0, metrics=[], expected=3 (0 + 1 + 2)
        (1, [], 6),          # prediction=1, metrics=[], expected=6 (1 + 2 + 3)
        (2, [], 9),          # prediction=2, metrics=[], expected=9 (2 + 3 + 4)
        (3, [lambda x: x * 2], 15),  # prediction=3, metrics=[lambda x: x * 2], expected=15 (6 + 3 + 4 + 5)
        (0, [lambda x: x + 10], 33), # prediction=0, metrics=[lambda x: x + 10], expected=33 (10 + 0 + 1 + 2)
        (1, [lambda x: x * 3], 12),  # prediction=1, metrics=[lambda x: x * 3], expected=12 (3 + 1 + 2 + 3)
        (2, [lambda x: x - 1], 9)    # prediction=2, metrics=[lambda x: x - 1], expected=9 (1 + 2 + 3 + 4)
    ]

    # Executar os testes
    for i, (prediction, metrics, expected) in enumerate(test_cases):
        result = get_sum_metrics(prediction, metrics)
        if result == expected:
            print(f"Caso de teste {i+1} ✅passou: esperado {expected}, obtido {result}")
        else:
            print(f"Caso de teste {i+1} ❌falhou: esperado {expected}, obtido {result}")

# Executar a função de teste
test_get_sum_metrics()

Caso de teste 1 ✅passou: esperado 3, obtido 3
Caso de teste 2 ✅passou: esperado 6, obtido 6
Caso de teste 3 ✅passou: esperado 9, obtido 9
Caso de teste 4 ❌falhou: esperado 15, obtido 18
Caso de teste 5 ❌falhou: esperado 33, obtido 13
Caso de teste 6 ❌falhou: esperado 12, obtido 9
Caso de teste 7 ❌falhou: esperado 9, obtido 10
